## Nodebook to test REDIS vectorstore queries

In [3]:
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)
from dotenv import load_dotenv
import os
import json

from pandas import DataFrame, to_datetime, read_parquet
from numpy import array, float32
import openai
from datetime import datetime, date
from typing import List

In [2]:
if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


In [6]:
REDIS_PASSWORD = os.environ.get('REDIS_PASSWORD')

In [7]:
# Connect to Redis
redis_client = redis.Redis(
    host="redis-17123.c56.east-us.azure.cloud.redislabs.com",
    port=17123,
    password=REDIS_PASSWORD
)
redis_client.ping()

True

In [16]:
def search_redis(
    redis_client: redis.Redis,
    user_query: str,
    index_name: str = "openai_embeddings-index",
    vector_field: str = "content_ada_embedding",
    return_fields: list = ['subject', 'web_link', 'content2embed', "vector_score"],
    hybrid_fields = "*",
    k: int = 20,
    print_results: bool = True,
) -> List[dict]:

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(input=user_query,
                                            model="text-embedding-ada-002",
                                            )["data"][0]['embedding']

    # Prepare the Query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'
    query = (
        Query(base_query)
         .return_fields(*return_fields)
         .sort_by("vector_score")
         .paging(0, k)
         .dialect(2)
    )
    params_dict = {"vector": array(embedded_query).astype(dtype=float32).tobytes()}

    # perform vector search
    results = redis_client.ft(index_name).search(query, params_dict)
    if print_results:
        for i, article in enumerate(results.docs):
            score = 1 - float(article.vector_score)
            print(f"{i}. {article.subject} {article.web_link} (Score: {round(score ,3) })")
    return results.docs

In [ ]:
# For using OpenAI to generate query embedding
results = search_redis(redis_client, 'Zielbild', vector_field='content_ada_embedding', k=10)